In [ ]:
# --- THIS IS THE STUB FOR AZURE OPENAI ---
# from openai import AzureOpenAI

# client = AzureOpenAI(
#   azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
#   api_key=os.getenv("AZURE_OPENAI_API_KEY"),
#   api_version="2024-02-01" # Or your desired API version
# )

# # In Azure, you use your "deployment name" instead of a model name
# MODEL = "your-deployment-name-for-gpt-4o" # e.g., "gpt-4o-deployment"
# ---

Part 1: The Basic Agent Loop
An agent is fundamentally a loop. It calls the model, checks if the model wants to use a tool, executes the tool if needed, and repeats until it gets a final answer.

Let's build this core loop.

In [6]:
# Cell 2: Configure your API Key
from openai import OpenAI, AsyncOpenAI
import os
import json
import inspect
from dotenv import load_dotenv, find_dotenv

# Create a .env file in the same directory and add your key:
# OPENAI_API_KEY="sk-...C2sA" (replace with your full key)
load_dotenv()

# Configure the OpenAI client
# It will automatically look for the OPENAI_API_KEY environment variable.
try:
    client = OpenAI()
    client.models.list()
    print("✅ OpenAI client initialized successfully.")
except Exception as e:
    print(f"🔴 Error initializing OpenAI client: {e}")
    print("Please make sure your OPENAI_API_KEY is set correctly in a .env file.")

# Let's use a fast and capable model for this demo
MODEL = "gpt-4o-mini"

✅ OpenAI client initialized successfully.


Part 2: The Core Agent Class & Basic Loop
This is the foundation. We'll create a reusable Agent class that can handle the conversation loop, format tools for the API, and execute function calls. We'll make the tool schema generation more robust using Python's inspect library.

In [7]:
# Cell 3: The Advanced Agent Class

def get_python_type(py_type):
    """Converts Python type hints to JSON schema types."""
    if py_type is int:
        return "integer"
    if py_type is str:
        return "string"
    if py_type is bool:
        return "boolean"
    if py_type is float:
        return "number"
    return "string"

class Agent:
    """A more robust agent that can use tools with the OpenAI API."""
    def __init__(self, client, model, system_prompt="", tools=None):
        self.client = client
        self.model = model
        # Use a dictionary for faster tool lookups
        self.tools = {t.__name__: t for t in tools} if tools else {}
        self.messages = [{"role": "system", "content": system_prompt}] if system_prompt else []

    def _create_tool_schema(self):
        """Dynamically creates the JSON schema for tools using inspection."""
        if not self.tools:
            return None
        
        tool_schemas = []
        for func_name, func in self.tools.items():
            sig = inspect.signature(func)
            parameters = {
                "type": "object",
                "properties": {},
                "required": [],
            }
            for name, param in sig.parameters.items():
                parameters["properties"][name] = {
                    "type": get_python_type(param.annotation),
                    "description": "", # Descriptions would require more complex parsing
                }
                if param.default is inspect.Parameter.empty:
                    parameters["required"].append(name)
            
            tool_schemas.append({
                "type": "function",
                "function": {
                    "name": func_name,
                    "description": func.__doc__,
                    "parameters": parameters,
                }
            })
        return tool_schemas

    def chat(self, user_input: str):
        """Runs the main agent loop."""
        print(f"👤 User: {user_input}")
        self.messages.append({"role": "user", "content": user_input})
        
        tool_schema = self._create_tool_schema()
        
        while True:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.messages,
                tools=tool_schema,
                tool_choice="auto"
            )
            response_message = response.choices[0].message
            
            # Check if the model wants to call one or more tools
            if response_message.tool_calls:
                self.messages.append(response_message)
                
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    print(f"⚙️ Model wants to call tool: {function_name}")
                    
                    function_to_call = self.tools.get(function_name)
                    function_args = json.loads(tool_call.function.arguments)
                    function_response = function_to_call(**function_args)
                    
                    self.messages.append({
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    })
            else:
                final_answer = response_message.content
                print(f"🤖 Agent: {final_answer}\n")
                self.messages.append({"role": "assistant", "content": final_answer})
                return final_answer

# --- Test the Basic Loop ---
def get_weather(location: str):
    """Returns the current weather for a given location."""
    if "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "15°C", "condition": "Cloudy"})
    else:
        return json.dumps({"location": location, "temperature": "22°C", "condition": "Clear"})

weather_agent = Agent(client, model=MODEL, tools=[get_weather])
weather_agent.chat("What's the weather like in Paris?")

👤 User: What's the weather like in Paris?


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
# Cell 4: Implement and Test Memory Tools

MEMORY_FILE = "memory_bank.json"

def add_to_memory(memory_text: str):
    """
    Adds a piece of factual information to the agent's long-term memory.
    Use this when the user states a preference or a fact about themselves.
    """
    print(f"🧠 Adding to memory: '{memory_text}'")
    if not os.path.exists(MEMORY_FILE):
        memories = []
    else:
        with open(MEMORY_FILE, 'r') as f:
            memories = json.load(f)
    
    memories.append(memory_text)
    with open(MEMORY_FILE, 'w') as f:
        json.dump(memories, f, indent=2)
    return f"OK, I've remembered that."

def get_all_memory():
    """Retrieves all facts currently stored in the agent's long-term memory."""
    print("🧠 Retrieving all memories...")
    if not os.path.exists(MEMORY_FILE):
        return "I don't have any memories stored yet."
    with open(MEMORY_FILE, 'r') as f:
        return json.dumps(json.load(f))

# --- Test the agent with memory ---
if os.path.exists(MEMORY_FILE):
    os.remove(MEMORY_FILE)

memory_agent = Agent(client, model=MODEL, tools=[add_to_memory, get_all_memory])
memory_agent.chat("Please remember that my user ID is 789-Alpha.")
memory_agent.chat("What is my user ID?")

In [ ]:
# Cell 5: Implement and Test Delegation

def delegate_to_smarter_model(task_description: str):
    """
    Use this for very difficult or complex creative tasks.
    A more capable specialist model will handle it.
    """
    print(f"🔥 Delegating complex task: '{task_description}'")
    
    # Here you could use a more powerful model like "gpt-4o"
    delegation_client = OpenAI()
    response = delegation_client.chat.completions.create(
        model="gpt-4o", # Using the full gpt-4o for the hard task
        messages=[{"role": "user", "content": task_description}]
    )
    return response.choices[0].message.content

# --- Test the delegation agent ---
delegation_agent = Agent(client, model=MODEL, tools=[delegate_to_smarter_model])
delegation_agent.chat("This is a hard task: write a sonnet about the challenges of AI alignment.")

In [ ]:
# Cell 6: Implement and Test Asynchronous Tasks
import asyncio
import uuid
import nest_asyncio

# Allows asyncio to run in a Jupyter Notebook
nest_asyncio.apply()

# Global dict to act as our task database
tasks = {}

async def long_running_model_call(description: str, task_id: str):
    """Simulates a long-running background task."""
    print(f"⏳ Task {task_id} started in background.")
    tasks[task_id]['status'] = 'running'
    await asyncio.sleep(10) # Simulate network latency and processing time
    
    async_client = AsyncOpenAI()
    response = await async_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": description}]
    )
    
    tasks[task_id]['status'] = 'complete'
    tasks[task_id]['result'] = response.choices[0].message.content
    print(f"✅ Task {task_id} is complete.")

def create_background_task(description: str):
    """Creates a new background task and immediately returns a task ID."""
    task_id = str(uuid.uuid4())[:8]
    tasks[task_id] = {'status': 'pending'}
    asyncio.create_task(long_running_model_call(description, task_id))
    return f"Task created with ID: {task_id}. Check its status later."

def check_task_status(task_id: str):
    """Checks the status of a previously created background task."""
    task = tasks.get(task_id)
    if not task:
        return f"Error: No task found with ID {task_id}"
    
    if task['status'] == 'complete':
        return f"Task {task_id} is complete. Result: {task['result']}"
    else:
        return f"Task {task_id} is still {task['status']}."

# --- Run the async demo ---
async def run_async_demo():
    # Note: The agent itself doesn't need to be async, only the tool execution
    async_agent = Agent(client, model=MODEL, tools=[create_background_task, check_task_status])
    
    task_id_response = async_agent.chat("Create a background task to write a short story about a robot who discovers music.")
    task_id = task_id_response.split(":")[-1].strip()
    
    async_agent.chat("While that's running, what's the weather in Mumbai?") # Fails, as it lacks the tool!
    
    print("\n--- Waiting 12 seconds for the task to complete... ---\n")
    await asyncio.sleep(12)
    
    async_agent.chat(f"What's the status of my task {task_id}?")

# Run the demo
asyncio.run(run_async_demo())

In [ ]:
# Cell 7: Implement and Test Dynamic Tool Creation

# The agent's tools will now be a dictionary that we can modify
dynamic_tool_set = {}

def add_tool(function_name: str, python_code: str):
    """
    DANGEROUS: Creates a new tool from a string of Python code and adds it to the agent's toolkit.
    """
    print(f"✨ Attempting to create new tool: '{function_name}'")
    try:
        local_scope = {}
        # DANGEROUS: Execute the user-provided code
        exec(python_code, globals(), local_scope)
        
        # Add the newly created function to our dynamic toolset
        dynamic_tool_set[function_name] = local_scope[function_name]
        
        # We need to re-initialize the agent to make it aware of the new tool
        # In a more complex system, you might update the agent's state directly
        global bootstrap_agent 
        bootstrap_agent = Agent(client, model=MODEL, tools=list(dynamic_tool_set.values()))

        print(f"✅ Successfully created tool: '{function_name}'. Agent has been updated.")
        return f"Tool '{function_name}' has been added."
    except Exception as e:
        print(f"🔴 Failed to create tool: {e}")
        return f"Error creating tool: {e}"

# --- Test the self-bootstrapping agent ---
# Start the agent with only the 'add_tool' capability
dynamic_tool_set['add_tool'] = add_tool
bootstrap_agent = Agent(client, model=MODEL, tools=list(dynamic_tool_set.values()))

# Step 1: Teach the agent a new trick
bootstrap_agent.chat("""
Please create a new tool for me. Its name should be 'calculate_circle_area'.
The python code is:
def calculate_circle_area(radius: float):
    \"\"\"Calculates the area of a circle given its radius.\"\"\"
    return 3.14159 * (radius ** 2)
""")

# Step 2: Use the newly created tool
bootstrap_agent.chat("Excellent. Now use the new tool to find the area of a circle with a radius of 10.")